<a href="https://colab.research.google.com/github/anilbhatt1/EVA4_Phase1/blob/master/EVA4_Session_2_With_explanations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [7]:
# # [filter size, stride, padding]
# #Assume the two dimensions are the same
# #Each kernel requires the following parameters:
# # - k_i: kernel size
# # - s_i: stride
# # - p_i: padding (if padding is uneven, right padding will higher than left padding; "SAME" option in tensorflow)
# # 
# #Each layer i requires the following parameters to be fully represented: 
# # - n_i: number of feature (data layer has n_1 = imagesize )
# # - j_i: distance (projected to image pixel distance) between center of two adjacent features
# # - r_i: receptive field of a feature in layer i
# # - start_i: position of the first feature's receptive field in layer i (idx start from 0, negative means the center fall into padding)

# import math
# convnet =   [[11,4,0],[3,2,0],[5,1,2],[3,2,0],[3,1,1],[3,1,1],[3,1,1],[3,2,0],[6,1,0], [1, 1, 0]]
# layer_names = ['conv1','pool1','conv2','pool2','conv3','conv4','conv5','pool5','fc6-conv', 'fc7-conv']
# imsize = 227

# def outFromIn(conv, layerIn):
#   n_in = layerIn[0]
#   j_in = layerIn[1]
#   r_in = layerIn[2]
#   start_in = layerIn[3]
#   k = conv[0]
#   s = conv[1]
#   p = conv[2]
  
#   n_out = math.floor((n_in - k + 2*p)/s) + 1
#   actualP = (n_out-1)*s - n_in + k 
#   pR = math.ceil(actualP/2)
#   pL = math.floor(actualP/2)
  
#   j_out = j_in * s
#   r_out = r_in + (k - 1)*j_in
#   start_out = start_in + ((k-1)/2 - pL)*j_in
#   return n_out, j_out, r_out, start_out
  
# def printLayer(layer, layer_name):
#   print(layer_name + ":")
#   print("\t n features: %s \n \t jump: %s \n \t receptive size: %s \t start: %s " % (layer[0], layer[1], layer[2], layer[3]))
 
# layerInfos = []
# if __name__ == '__main__':
# #first layer is the data layer (image) with n_0 = image size; j_0 = 1; r_0 = 1; and start_0 = 0.5
#   print ("-------Net summary------")
#   currentLayer = [imsize, 1, 1, 0.5]
#   printLayer(currentLayer, "input image")
#   for i in range(len(convnet)):
#     currentLayer = outFromIn(convnet[i], currentLayer)
#     layerInfos.append(currentLayer)
#     printLayer(currentLayer, layer_names[i])
#   print ("------------------------")
#   layer_name = raw_input ("Layer name where the feature in: ")
#   layer_idx = layer_names.index(layer_name)
#   idx_x = int(raw_input ("index of the feature in x dimension (from 0)"))
#   idx_y = int(raw_input ("index of the feature in y dimension (from 0)"))
  
#   n = layerInfos[layer_idx][0]
#   j = layerInfos[layer_idx][1]
#   r = layerInfos[layer_idx][2]
#   start = layerInfos[layer_idx][3]
#   assert(idx_x < n)
#   assert(idx_y < n)
  
#   print ("receptive field: (%s, %s)" % (r, r))
#   print ("center: (%s, %s)" % (start+idx_x*j, start+idx_y*j))

In [13]:
class Net(nn.Module):
    def __init__(self):
        # super(Net, self).__init__()
        # self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input - 28x28x1  Output - 28x28x32 : RF -> 3
        # self.conv2 = nn.Conv2d(32, 64, 3, padding=1) # i/p - 28x28x32 o/p - 28x28x64 : RF -> 5
        # self.pool1 = nn.MaxPool2d(2, 2) # i/p - 28x28x64 o/p 14x14x64 : RF -> 6
        # self.conv3 = nn.Conv2d(64, 128, 3, padding=1) # i/p 14x14x64 o/p 14x14x128 : RF -> 10
        # self.conv4 = nn.Conv2d(128, 256, 3, padding=1) # i/p 14x14x128 o/p 14x14x256 : RF -> 14
        # self.pool2 = nn.MaxPool2d(2, 2)  # i/p 14x14x256 o/p 7x7x256   : RF -> 16
        # self.conv5 = nn.Conv2d(256, 512, 3)  # i/p 7x7x256 o/p 5x5x512 -> RF 24
        # self.conv6 = nn.Conv2d(512, 10, 5) # i/p 5x5x512 o/p 1x1x10 -> RF 40
        # self.conv7 = nn.Conv2d(1024, 10, 3)  # i/p 3x3x1024 o/p 1x1x10 -> RF 40
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF  #input - 28x28x1  Output - 28x28x32 : RF -> 3
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)  # i/p - 28x28x32 o/p - 28x28x64 : RF -> 5
        self.pool1 = nn.MaxPool2d(2, 2)  # i/p - 28x28x64 o/p 14x14x64 : RF -> 6
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) # i/p 14x14x64 o/p 14x14x128 : RF -> 10
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)  # i/p 14x14x128 o/p 14x14x256 : RF -> 14
        self.pool2 = nn.MaxPool2d(2, 2) ## i/p 14x14x256 o/p 7x7x256   : RF -> 16
        self.conv5 = nn.Conv2d(256, 512, 3) ## i/p 7x7x256 o/p 5x5x512 -> RF 24
        self.conv6 = nn.Conv2d(512, 1024, 3) ## i/p 5x5x512 o/p 3x3x1024 -> RF 26
        self.conv7 = nn.Conv2d(1024, 10, 3)   ## i/p 3x3x1024 o/p 1x1x10 -> RF 28     

    def forward(self, x):
      x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
      x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
      x = F.relu(self.conv6(F.relu(self.conv5(x))))
      # x = F.relu(self.conv7(x))  ## THIS IS THE ONLY CHANGE I HAD TO COMMENT TO IMPROVE PERFORMANCE FROM 28% TO 98%
      x = self.conv7(x)  ## ## THIS IS THE ONLY CHANGE I HAD TO ADD TO IMPROVE PERFORMANCE FROM 28% TO 98%
      x = x.view(-1, 10)
        # x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        # x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        # x = F.relu(self.conv6(F.relu(self.conv5(x))))
        # x = F.relu(self.conv5(x))
        # x = F.relu(self.conv7(x))
        # x = self.conv7(x)
        # x = self.conv6(x)
        # x = x.view(-1, 10)
      return F.log_softmax(x)

In [14]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [15]:
torch.manual_seed(1)  # This is to generate random numbers
batch_size = 128

# num_workers -> which denotes the number of processes that generate batches in parallel. A high enough number of workers assures that 
# CPU computations are efficiently managed. 0 means data will be loaded in the main process.
# pin_memory -> If you load your samples in the Dataset on CPU and would like to push it during training to the GPU, you can speed up 
# the host to device transfer by enabling pin_memory . This lets your DataLoader allocate the samples in page-locked memory, which 
# speeds-up the transfer. 

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(), # Will transform (H,W,C) to (C,W,H) and scale between 0-1 # Converts a PIL Image or numpy.ndarray (H x W x C) in the range [0, 255] to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1
                        transforms.Normalize((0.1307,), (0.3081,))    # 0.1307 - MNIST mean, 0.3081 - MNIST std
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [16]:
from tqdm import tqdm  

# Instantly make your loops show a smart progress meter - just wrap any iterable with tqdm(iterable), and you're done!

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates 
        # the gradients on subsequent backward passes
        output_train = model(data)
        loss = F.nll_loss(output_train, target)  # Negative log likehood. Loss calculated between prediction from model & input target
        loss.backward()  # Operation that calculates gradient (back-prop)
        optimizer.step() # Operation that updates the weights (back-prop)
        pbar.set_description(desc= f'ding loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():  # We are disabling gradient calculation as we are dealing with test data
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output_test = model(data)
            test_loss += F.nll_loss(output_test, target, reduction='sum').item()  # sum up batch loss
            pred = output_test.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            # .eq -> Computes element-wise equality between values in pred and target. 
            # To enable comparison we are changing target to same shape as pred using target.view_as(pred)
            # Output of pred.eq(target.view_as(pred)) will be 1 (True) or 0(False)
            # We will sum-up this with 'correct' for each of test row
            # item() will help to convert output of pred.eq(target.view_as(pred)).sum() to a proper number
            # Basically we are trying to get count of correctly predicted records from test dataset
            # We will use this 'correct' to predict accuracy i.e. accuracy = correct/len(test_loader.dataset)

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [17]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

ding loss=0.07875696569681168 batch_id=468: 100%|██████████| 469/469 [00:39<00:00, 11.76it/s]



Test set: Average loss: 0.0658, Accuracy: 9791/10000 (98%)

